**Importing the libaries for the Moddel**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score

**Loading the Dataset**

In [2]:
df = pd.read_csv('refined_data_with_features.csv')

In [3]:
df.head(6)

,percentage yield,gravity,vapour pressure,ten percent distillation point,fraction end point,boiling_range,yield_to_gravity,pressure_to_gravity
0,6.9,38.4,6.1,220,235,15,0.179688,0.158854
1,14.4,40.3,4.8,231,307,76,0.357320,0.119107
2,7.4,40.0,6.1,217,212,-5,0.185000,0.152500
3,8.5,31.8,0.2,316,365,49,0.267296,0.006289
4,8.0,40.8,3.5,210,218,8,0.196078,0.085784
5,2.8,41.3,1.8,267,235,-32,0.067797,0.043584


**Model preprocessing**

In [4]:
x = df[["percentage yield", "gravity", "vapour pressure", "ten percent distillation point", "fraction end point", "yield_to_gravity", "pressure_to_gravity"]]  # Independent variables
y = df["boiling_range"]  # Dependent variable

In [5]:
# Splitting the data into the training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
# Scale the data (for models sensitive to feature scaling)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

**Training the model**

In [7]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(x_train, y_train)

DecisionTreeRegressor(random_state=42)

**Prediction**

In [8]:
y_pred = dt_model.predict(x_test)

**Model evaluation**

In [9]:
# Regression Metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Regression Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Regression Metrics:
Mean Squared Error (MSE): 182.14285714285714
R-squared (R2): 0.9134822311405805


In [10]:
import pickle

In [11]:
file = 'boiling_range_model'
pickle.dump(dt_model, open(file, 'wb'))


In [21]:


import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
import pickle
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)

# Load the trained model
model = pickle.load(open('boiling_range_model', 'rb'))

# Load the scaler (if used during training)
# scaler = StandardScaler() # Assuming you saved the scaler as well
# scaler = pickle.load(open('scaler.pkl', 'rb')) # Load if you saved the scaler

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()

        # Ensure the required features are present in the input data
        required_features = ["percentage yield", "gravity", "vapour pressure", "ten percent distillation point", "fraction end point", "yield_to_gravity", "pressure_to_gravity"]
        for feature in required_features:
            if feature not in data:
                return jsonify({"error": f"Missing required feature: {feature}"}), 400

        # Create a DataFrame from the input data
        input_df = pd.DataFrame([data])

        # Scale the input data using the same scaler used during training
        # input_scaled = scaler.transform(input_df) # Uncomment if you used a scaler

        # Make prediction
        prediction = model.predict(input_df)

        return jsonify({"prediction": prediction.tolist()})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [22]:
import requests

url = "http://127.0.0.1:5000/predict"
headers = {"Content-Type": "application/json"}
data = {
    "percentage yield": 90.5,
    "gravity": 0.87,
    "vapour pressure": 12.5,
    "ten percent distillation point": 150,
    "fraction end point": 240,
    "yield_to_gravity": 104.2,
    "pressure_to_gravity": 14.36
}

response = requests.post(url, json=data, headers=headers)
print(response.json())


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023C1B7B3F10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [23]:
print("Hello there")

Hello there
